In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import musicbrainzngs

In [ ]:
# on Spotify dev site, create new app to find client_id and client_secret id

client_id = ""
client_secret = ""
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [ ]:
# check API throttling issue

sp.me()  # This fetches your account info and is a quick way to check API access.

HTTP Error for GET to https://api.spotify.com/v1/me/ with Params: {} returned 401 due to Valid user authentication required


SpotifyException: http status: 401, code: -1 - https://api.spotify.com/v1/me/:
 Valid user authentication required, reason: None

In [ ]:
playlist_id = "3pME5KrheLTzGFpX9yaLZC" # Billboard Year End Hot 100 1980s


In [53]:
all_data = []

In [ ]:
def get_playlist_data(playlist_id):

    track_data = []
    playlist = sp.playlist_tracks(playlist_id)

    for item in playlist["items"]:
        track = item["track"]
        track_name = track["name"]
        artist_name = ", ".join([artist["name"] for artist in track["artists"]])

        genres = []
        for artist in track["artists"]:
            artist_info = sp.artist(artist["id"])
            if artist_info["genres"]:
                genres.extend(artist_info["genres"])

        genres = ", ".join(set(genres)) if genres else "Unknown"
        track_data.append([track_name, artist_name, genres])

    return track_data

In [ ]:
track_from_80s = get_playlist_data(playlist_id)

NameError: name 'get_playlist_data' is not defined

# Scrape from MusicBrainz for Artist Genres

Only half of Spotify artist genre tags returned with value, so scrape rest of information from MusicBrainz by finding artist ID, sorting by "count" (defined by the number of users that tagged that artist with that specific tag), and attach #1 tag. For multiple artists on track, save #1 tag per artist.

In [32]:
musicbrainzngs.set_useragent("TrendScraper", "1.0", "ishiharay15@gmail.com")

In [ ]:
def get_artist_genre(artist_name):
    try:

        result = musicbrainzngs.search_artists(artist=artist_name, limit=1)

        if "artist-list" in result and len(result["artist-list"]) > 0:
            artist = result["artist-list"][0]
            artist_id = artist["id"]

            artist_info = musicbrainzngs.get_artist_by_id(artist_id, includes=["tags"])

            # Extract genres with counts
            tag_list = artist_info.get("artist", {}).get("tag-list", [])
            sorted_tags = sorted(tag_list, key=lambda x: int(x["count"]), reverse=True) if tag_list else []

            genres = [tag["name"] for tag in sorted_tags]

            return ", ".join(genres) if genres else "Unknown"

        return "Unknown"
    
    except Exception as e:
        print(f"Error fetching genre for {artist_name}: {e}")
        return "Unknown"

In [ ]:
def update_unknown_genres(df):
    
    for index, row in df.iterrows():
        if row["Genres"] == "Unknown":
            artist_names = row["Artists"].split(", ")
            genre_list = []

            for artist_name in artist_names:
                genres = get_artist_genre(artist_name)
                top_genre = genres.split(", ")[0] if genres != "Unknown" else "Unknown"

                if top_genre != "Unknown":
                    genre_list.append(top_genre)

            if genre_list:
                df.at[index, "Genres"] = ", ".join(genre_list)

    return df


In [51]:
df

,Track Name,Artists,Genres
0,Too Close,Next,r&b
1,The Boy Is Mine,"Brandy, Monica",r&b
2,You're Still The One,Shania Twain,country
3,Truly Madly Deeply,Savage Garden,pop
4,How Do I Live,LeAnn Rimes,country
...,...,...,...
92,4 Seasons Of Loneliness,Boyz II Men,"new jack swing, quiet storm, r&b"
93,Father,LL COOL J,east coast hip hop
94,Thinkin' Bout It,Gerald Levert,"quiet storm, smooth r&b"
95,Nobody's Supposed to Be Here,Deborah Cox,2008 universal fire victim


In [52]:
unknown_count = (df["Genres"] == "Unknown").sum()
print(f"Number of tracks with unknown genre: {unknown_count}")

Number of tracks with unknown genre: 1


# Scrape Year End Hot 100 Account for all playlists 1946 - 2024

CSV with all playlist titles and IDs saved at "year_end_hot_100_playlists.csv"

In [ ]:
def get_user_playlists(user_id):

    playlists = []
    results = sp.user_playlists(user_id)

    while results:
        for playlist in results["items"]:
            playlists.append({
                "name": playlist["name"],
                "id": playlist["id"],
                "tracks": playlist["tracks"]["total"],
                "snapshot_id": playlist["snapshot_id"]
            })

        results = sp.next(results) if results["next"] else None

    return playlists

user_id = "31zq3uw3m45cxpbmhsuctbph7ucm"
playlists = get_user_playlists(user_id)


for p in playlists:
    print(f"Playlist: {p['name']} (ID: {p['id']}, Tracks: {p['tracks']})")

df = pd.DataFrame(playlists)



Playlist: 2024 Billboard Year End Hot 100 (ID: 1LSXxWhYAVWWfb847gLIcD, Tracks: 100)
Playlist: 2023 Billboard Year End Hot 100 (ID: 2tpSdD4wfe5elx7iMtxDcF, Tracks: 100)
Playlist: 2022 Billboard Year End Hot 100 (ID: 1P39I532Z3TWDw2ggm3Y7k, Tracks: 100)
Playlist: 2021 Billboard Year End Hot 100 (ID: 5pT9DvpEMYtkINr2vOtwmQ, Tracks: 100)
Playlist: 2020 Billboard Year End Hot 100 (ID: 7iOLp82ZbOu5I04TrOBeru, Tracks: 100)
Playlist: 2019 Billboard Year End Hot 100 (ID: 60uWnDLPSMaOXrJOmm1BCn, Tracks: 100)
Playlist: 2018 Billboard Year End Hot 100 (ID: 3fG7cr6o7osDoorF7qnb1g, Tracks: 100)
Playlist: 2017 Billboard Year End Hot 100 (ID: 0mxbQZjOVjC8xFuqBExatn, Tracks: 100)
Playlist: 2016 Billboard Year End Hot 100 (ID: 1KewXQLrI99SJB1fQwx8Sp, Tracks: 100)
Playlist: 2015 Billboard Year End Hot 100 (ID: 6tYE3NllVJ1yRxZPWkMhAU, Tracks: 100)
Playlist: 2014 Billboard Year End Hot 100 (ID: 1qaEDj0b6V16jR197rBzmR, Tracks: 100)
Playlist: 2013 Billboard Year End Hot 100 (ID: 3MOm2n36QOgjHQFJgShW1U, Track

In [7]:
df = df[df["tracks"] >= 90]

In [8]:
output_file = f"year_end_hot_100_playlists.csv"
df.to_csv(output_file, index=False)